In [6]:
import grpc
from apis import PredictRequest, PredictionService, ModelSpec

SyntaxError: invalid syntax (apis.py, line 61)

In [2]:
from base import Message

In [3]:
def unwrap_pb(obj):
    if isinstance(obj, tuple(Message.__subclasses__())):
        return obj._protobuf
    return obj

def wrap_pb(pb, cls):
    return cls()._protobuf

In [5]:
model_spec = ModelSpec(name='mnist', version=2, signature_name='predict_images')

NameError: name 'ModelSpec' is not defined

In [ ]:
model

In [44]:
type(model_spec)a

apis.ModelSpec

In [45]:
msp = model_spec._protobuf

In [46]:
msp_w = wrap_pb(msp, ModelSpec)

In [47]:
Message.__subclasses__()

[sources.ServableVersionPolicy,
 config.ModelConfig,
 apis.ModelSpec,
 apis.PredictRequest,
 apis.PredictResponse]

In [48]:
x = unwrap_pb(msp_w)

In [52]:
x.ByteSize()

0

In [3]:
from __future__ import print_function
import sys
import threading

In [4]:
import grpc
import numpy
import tensorflow as tf
import mnist_input_data

In [5]:
test_data_set = mnist_input_data.read_data_sets('/tmp').test

W0819 23:56:05.885027 140626752575296 deprecation.py:323] From <ipython-input-5-17e66ce9737f>:1: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0819 23:56:05.888272 140626752575296 deprecation.py:323] From /home/jagan/.virtualenvs/tfs-client-env/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0819 23:56:05.892272 140626752575296 deprecation.py:323] From /home/jagan/.virtualenvs/tfs-client-env/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is depre

Extracting /tmp/train-images-idx3-ubyte.gz


W0819 23:56:06.137973 140626752575296 deprecation.py:323] From /home/jagan/.virtualenvs/tfs-client-env/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0819 23:56:06.180411 140626752575296 deprecation.py:323] From /home/jagan/.virtualenvs/tfs-client-env/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Extracting /tmp/train-labels-idx1-ubyte.gz
Extracting /tmp/t10k-images-idx3-ubyte.gz
Extracting /tmp/t10k-labels-idx1-ubyte.gz


In [7]:
predict_request = PredictRequest(model_spec=model_spec)
prediction_service = PredictionService('0.0.0.0:8500', 10)
num_tests = 100

In [8]:
for _ in range(2):
    image, label = test_data_set.next_batch(1)
    predict_request.inputs = {'images' : image[0].reshape(1,-1)}
    reponse = prediction_service.predict(predict_request)    

In [9]:
reponse.parse_outputs()

{'scores': array([[1.0906742e-06, 6.7125021e-12, 5.8659766e-10, 1.4442198e-03,
         3.8212602e-06, 1.4825355e-05, 5.5803252e-08, 3.6779717e-01,
         1.7156720e-03, 6.2902308e-01]], dtype=float32)}

In [10]:
response = reponse.parse_outputs()

In [11]:
response['scores'].shape

(1, 10)

In [12]:
predict_request.byte_size

3195

In [ ]:
tf.app.flags.DEFINE_integer('concurrency', 1,
                            'maximum number of concurrent inference requests')
tf.app.flags.DEFINE_integer('num_tests', 100, 'Number of test images')
tf.app.flags.DEFINE_string('server', '', 'PredictionService host:port')
tf.app.flags.DEFINE_string('work_dir', '/tmp', 'Working directory. ')
FLAGS = tf.app.flags.FLAGS


class _ResultCounter(object):
  """Counter for the prediction results."""

  def __init__(self, num_tests, concurrency):
    self._num_tests = num_tests
    self._concurrency = concurrency
    self._error = 0
    self._done = 0
    self._active = 0
    self._condition = threading.Condition()

  def inc_error(self):
    with self._condition:
      self._error += 1

  def inc_done(self):
    with self._condition:
      self._done += 1
      self._condition.notify()

  def dec_active(self):
    with self._condition:
      self._active -= 1
      self._condition.notify()

  def get_error_rate(self):
    with self._condition:
      while self._done != self._num_tests:
        self._condition.wait()
      return self._error / float(self._num_tests)

  def throttle(self):
    with self._condition:
      while self._active == self._concurrency:
        self._condition.wait()
      self._active += 1


def _create_rpc_callback(label, result_counter):
  """Creates RPC callback function.

  Args:
    label: The correct label for the predicted example.
    result_counter: Counter for the prediction result.
  Returns:
    The callback function.
  """
  def _callback(result_future):
    """Callback function.

    Calculates the statistics for the prediction result.

    Args:
      result_future: Result future of the RPC.
    """
    exception = result_future.exception()
    if exception:
      result_counter.inc_error()
      print(exception)
    else:
      sys.stdout.write('.')
      sys.stdout.flush()
      response = numpy.array(
          result_future.result().outputs['scores'].float_val)
      prediction = numpy.argmax(response)
      if label != prediction:
        result_counter.inc_error()
    result_counter.inc_done()
    result_counter.dec_active()
  return _callback


def do_inference(hostport, work_dir, concurrency, num_tests):
  """Tests PredictionService with concurrent requests.

  Args:
    hostport: Host:port address of the PredictionService.
    work_dir: The full path of working directory for test data set.
    concurrency: Maximum number of concurrent requests.
    num_tests: Number of test images to use.

  Returns:
    The classification error rate.

  Raises:
    IOError: An error occurred processing test data set.
  """
  test_data_set = mnist_input_data.read_data_sets(work_dir).test
  channel = grpc.insecure_channel(hostport)
  stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)
  result_counter = _ResultCounter(num_tests, concurrency)
  for _ in range(num_tests):
    request = predict_pb2.PredictRequest()
    request.model_spec.name = 'mnist'
    request.model_spec.signature_name = 'predict_images'
    image, label = test_data_set.next_batch(1)
    request.inputs['images'].CopyFrom(
        tf.contrib.util.make_tensor_proto(image[0], shape=[1, image[0].size]))
    result_counter.throttle()
    result_future = stub.Predict.future(request, 5.0)  # 5 seconds
    result_future.add_done_callback(
        _create_rpc_callback(label[0], result_counter))
  return result_counter.get_error_rate()


def main(_):
  if FLAGS.num_tests > 10000:
    print('num_tests should not be greater than 10k')
    return
  if not FLAGS.server:
    print('please specify server host:port')
    return
  error_rate = do_inference(FLAGS.server, FLAGS.work_dir,
                            FLAGS.concurrency, FLAGS.num_tests)
  print('\nInference error rate: %s%%' % (error_rate * 100))


if __name__ == '__main__':
  tf.app.run()